# Cleaning Nashville Housing Data in SQL

In [ ]:
-- Viewing All Data
SELECT *
FROM ..NashvilleHousing


## 1.  Standardized Date Format

In [ ]:
-- Removing Time Stamp from Date (Serves No Purpose)
Select SaleDate, CONVERT(Date, SaleDate)
From ..NashvilleHousing



In [3]:
-- Updating Date Column
Update NashvilleHousing
Set SaleDate = Convert(Date,SaleDate)

(56373 rows affected)

Total execution time: 00:00:00.214

## 2\. Populating Missing Property Addresses

In [ ]:
-- Finding Where Property Addresses are Null
Select *
From housingData.dbo.NashvilleHousing
Where PropertyAddress is NULL

In [ ]:
-- Seeing if ParcelID can be used as a reference point to capture addresses
Select *
From housingData.dbo.NashvilleHousing
Order BY ParcelID

In [ ]:
-- Using ParcelID to populate Missing PropertyAddresses
SELECT a.parcelID, a.PropertyAddress, b.parcelID, b.PropertyAddress, ISNULL(a.PropertyAddress, b.PropertyAddress)
FROM ..NashvilleHousing a
JOIN ..NashvilleHousing b
    ON a.PARCELID = b.PARCELID
    AND a.UNIQUEID <> b. UNIQUEID
WHERE a.PropertyAddress is NULL

In [ ]:
-- Updating Missing Addresses in PropertyAddress Column
Update a
SET PropertyAddress = ISNULL(a.PropertyAddress, b.PropertyAddress)
FROM ..NashvilleHousing a
JOIN ..NashvilleHousing b
    ON a.PARCELID = b.PARCELID
    AND a.UNIQUEID <> b. UNIQUEID

After populating all of the missing addresses, the column can now be used to make accurate remarks about the locations of the houses for sale in Nashville. Previously with the missing data, results would be miss leading as it wouldn't account for all of the properties that were missing the data for that column. We can use the addresses of the houses for sale to see which locations have the most houses for sale. We can then find specific reasons to why these locations have the most houses for sale and see whether its because the location has higher demand, or if its because of factors that are preventing the houses from being sold in that area.

## 3\. Breaking Property Address into Individual Columns (Address, City)

In [ ]:
Select PropertyAddress
FROM ..NashvilleHousing

In [ ]:
-- Split The Address into seperate columns where there is a comma using Substring
SELECT
SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress)-1 ) AS Address
, SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress) +1, LEN(PropertyAddress)) AS City
FROM ..NashvilleHousing 

In [ ]:
--Adding New Columns for Address and City to Table
ALTER TABLE NashvilleHousing
Add PropertySplitAddress NVARCHAR(225);

ALTER TABLE NashvilleHousing
Add PropertySplitCity NVARCHAR(225);

UPDATE NashvilleHousing
SET PropertySplitAddress = SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress)-1 )

UPDATE NashvilleHousing
SET PropertySplitCity = SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress) +1, LEN(PropertyAddress))

## 4\. Splitting Owner Address into Individual Columns (Address, City, State)

In [ ]:
SELECT n.OwnerAddress
FROM ..NashvilleHousing n

In [ ]:
--Using Parsename to split Address into seperate Columns by Replacing commas with periods
Select
PARSENAME(REPLACE(OwnerAddress, ',', '.') , 3) AS Addres
, PARSENAME(REPLACE(OwnerAddress, ',', '.') , 2) AS City
, PARSENAME(REPLACE(OwnerAddress, ',', '.') , 1) As State
FROM ..NashvilleHousing 

In [ ]:
-- Adding new  columns to table
ALTER TABLE NashvilleHousing
Add OwnerSplitAddress NVARCHAR(225);
ALTER TABLE NashvilleHousing
Add OwnerSplitCity NVARCHAR(225);
ALTER TABLE NashvilleHousing
Add OwnerSplitState NVARCHAR(225);

UPDATE NashvilleHousing
SET OwnerSplitAddress = PARSENAME(REPLACE(OwnerAddress, ',', '.') , 3)
UPDATE NashvilleHousing
SET OwnerSplitCity = PARSENAME(REPLACE(OwnerAddress, ',', '.') , 2)
UPDATE NashvilleHousing
SET OwnerSplitState = PARSENAME(REPLACE(OwnerAddress, ',', '.') , 1)

## 5\. Updating Format in "Sold as Vacant" column (Chainging Y and N to Yes and No)

In [ ]:
Select Distinct(SoldAsVacant), Count(SoldAsVacant)
From ..NashvilleHousing
Group By SoldAsVacant
Order by 2 DESC


Select SoldAsVacant
,   CASE WHEN SoldAsVacant = 'Y' THEN 'YES'
        WHEN SoldAsVacant = 'N' THEN 'NO'
        ELSE SoldAsVacant
    END

From ..NashvilleHousing

In [ ]:
-- Updating "SoldAsVacant" Column
UPDATE NashvilleHousing
SET SoldAsVacant =  CASE WHEN SoldAsVacant = 'Y' THEN 'YES'
                    WHEN SoldAsVacant = 'N' THEN 'NO'
                    ELSE SoldAsVacant
                    END


Creating consistant formatting allows for easy and accurate displayment of which properties are sold as vacant, and which are not.

## 6\. Removing Duplicates

In [ ]:
-- Finding All the Duplicates in the Table by Using Partition to count rows that have exact same information
WITH RowNumCTE AS(
SELECT *,
    ROW_NUMBER() OVER (
        PARTITION BY ParcelID,
                    PropertyAddress,
                    SalePrice,
                    SaleDate,
                    LegalReference
                    ORDER BY 
                        UniqueID
    ) row_num
FROM ..NashvilleHousing
)
SELECT *
FROM RowNumCTE
Where row_num > 1
Order BY PropertyAddress

In [ ]:
--Deleting Duplicate Rows Found
WITH RowNumCTE AS(
SELECT *,
    ROW_NUMBER() OVER (
        PARTITION BY ParcelID,
                    PropertyAddress,
                    SalePrice,
                    SaleDate,
                    LegalReference
                    ORDER BY 
                        UniqueID
    ) row_num
FROM ..NashvilleHousing
)
Delete
FROM RowNumCTE
Where row_num > 1


Duplicate information can lead to business decisions that should not be made, and inaccuracy when analyzing the data. If an indivudal was to just view the number of properties for sale in a certain location, they could be misled by duplicate rows.  They could promise clients a property in a current location  thinking they have more avalaible than they actually do, leading them to be unable to meet the customers expectations. Also when creating specific filters, such as all properties above a certain sale price, or value, the results could be misleading due to the duplicates. Removing the dupliactes can help prevent these situations.

## 7\. Deleting Old Coulms that have been Replaced

In [ ]:

-- 7. Deleting Old Columns that have been Replaced
SELECT *
FROM ..NashvilleHousing

ALTER TABLE ..NashvilleHousing
DROP COLUMN OwnerAddress, PropertyAddress, TaxDistrict